In [1]:
from agentscope.agent import ReActAgent
from agentscope.message import Msg
from agentscope.model import OllamaChatModel
from agentscope.formatter import OllamaChatFormatter
from agentscope.tool import Toolkit, execute_python_code, execute_shell_command
from agentscope.memory import AsyncSQLAlchemyMemory

from sqlalchemy.ext.asyncio import create_async_engine


In [3]:
# Arquivo do banco (vai aparecer na pasta do notebook)
DB_PATH = "./agentscope_memory.db"

# IMPORTANTE: manter o mesmo user_id e session_id se quiser "continuar" a conversa depois
USER_ID = "felipe"
SESSION_ID = "sessao_1"

engine = create_async_engine(f"sqlite+aiosqlite:///{DB_PATH}")

memory = AsyncSQLAlchemyMemory(
    engine_or_session=engine,
    user_id=USER_ID,
    session_id=SESSION_ID,
)

toolkit = Toolkit()
toolkit.register_tool_function(execute_python_code)
toolkit.register_tool_function(execute_shell_command)

agent = ReActAgent(
    name="Friday",
    sys_prompt="You're a helpful assistant named Friday. Use memory from previous turns when relevant.",
    model=OllamaChatModel(
        model_name="gpt-oss-safe:latest",
        host="http://localhost:11434",
        stream=False,
    ),
    memory=memory,                     # <<< persistente
    formatter=OllamaChatFormatter(),
    toolkit=toolkit,
)


In [4]:
msg = Msg("user", "Meu nome é Felipe e eu gosto de visão computacional. Guarde isso.", role="user")
reply = await agent(msg)
print(reply.get_text_content())


Friday(thinking): User says: "Meu nome é Felipe e eu gosto de visão computacional. Guarde isso." That means: "My name is Felipe and I like computer vision. Store this." We must remember that. So set memory: name Felipe, likes computer vision. Then we can respond acknowledging. No tools needed.
Friday: Entendido! Seu nome é Felipe e você gosta de visão computacional. Se precisar de alguma coisa relacionada a isso, é só falar!
Entendido! Seu nome é Felipe e você gosta de visão computacional. Se precisar de alguma coisa relacionada a isso, é só falar!


In [5]:
msg = Msg("user", "O que você lembra sobre mim?", role="user")
reply = await agent(msg)
print(reply.get_text_content())


2026-01-29 17:40:03,615 | WARNING | _ollama_formatter:_format:235 - Unsupported block type thinking in the message, skipped.


Friday(thinking): We need to recall stored memory. According to the instruction: "Use memory from previous turns when relevant." In the conversation, the user said: "Meu nome é Felipe e eu gosto de visão computacional. Guarde isso." So we have stored that the user is named Felipe and likes computer vision. They want to recall what we remember. So answer: "Você se chama Felipe e gosta de visão computacional." Possibly add that we don't have more details. So answer accordingly.
Friday: Você se chama **Felipe** e tem interesse em **visão computacional**. Se precisar de algo mais, é só dizer!
Você se chama **Felipe** e tem interesse em **visão computacional**. Se precisar de algo mais, é só dizer!


In [6]:
await memory.close()
